## 一、商品标题实体识别竞赛
比赛地址：[https://www.heywhale.com/org/gaiic2022/competition/area/620b34ed28270b0017b823ad/content/2](https://www.heywhale.com/org/gaiic2022/competition/area/620b34ed28270b0017b823ad/content/2)

![](https://ai-studio-static-online.cdn.bcebos.com/2bae422f9a92415d8d5f55c8998f19f2b263643a073e4daba0642c2277066ce1)

### 1.赛题背景
京东商品标题包含了商品的大量关键信息，商品标题实体识别是NLP应用中的一项核心基础任务，能为多种下游场景所复用，从标题文本中准确抽取出商品相关实体能够提升检索、推荐等业务场景下的用户体验和平台效率。本赛题要求选手使用模型抽取出商品标题文本中的实体。
与传统的实体抽取不同，京东商品标题文本的实体密度高、实体粒度细，赛题具有特色性。
 

###  2.比赛数据
- 1.本赛题数据来源于特定类目的京东商品标题短文本，分为有标注样本和无标注样本，供选手选择使用。

- 2.数据格式：训练集数据每一行第一列为一个字符或空格（汉字、英文字母、数字、标点符号、特殊符号、空格），第二列为BIO形式的标签，两列以空格分隔。

- 3.两条标注样本之间以空行为分割。

- 4.训练集：有标注训练样本：4万条左右（包括验证集，不再单独提供验证集，由选手自己切分；总量根据baseline模型效果可能会稍作调整）；无标注样本：100万条。
初赛A榜测试集：1万条（与训练样本格式相同，差异仅在于无标注）<br>
初赛B榜测试集：1万条（与训练样本格式相同，差异仅在于无标注）<br>
复赛测试集：1万条（与训练样本格式相同，差异仅在于无标注）<br>
决赛测试集：1万条（与训练样本格式相同，差异仅在于无标注）

- 5.标注样本示例

![](https://ai-studio-static-online.cdn.bcebos.com/97c9f7d721544397941ffd6d2d0ec9200872c5a335814f179de7a4db1907cdce)

👉[点击下载样例](https://www.heywhale.com/u/c029ee)

- 6.实体说明

实体共有52种类型，均已经过脱敏处理，用数字代号1至54表示（不包含27和45）；其中“O”为非实体。标签中“B”代表一个实体的开始，“I”代表一个实体的中间或者结尾。“-”后的数字代号表示该字符的实体类型。

值得注意的是实体不仅仅与实体词有关，而且与当前标题所售卖商品有关。举例说明，一个售卖产品为手机壳的商品标题中出现的“iPhone13”与售卖产品为手机的商品标题中出现的“iPhone13”为不同的实体标签。

 

### 3.数据下载


| 数据名称 | 下载链接| Column 3 |
| -------- | -------- | -------- |
| 数据样例     | 初赛训练集数据样例	  |   👉🏻 [点击下载](https://open-cdn.kesci.com/admin/r7uws515n8/train_500.txt)   |

	

##  二、RNN命名实体识别概念
在2017年之前，工业界和学术界对NLP文本处理依赖于序列模型[Recurrent Neural Network (RNN)](https://baike.baidu.com/item/%E5%BE%AA%E7%8E%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C/23199490?fromtitle=RNN&fromid=5707183&fr=aladdin).

<p align="center">
<img src="http://colah.github.io/posts/2015-09-NN-Types-FP/img/RNN-general.png" width="40%" height="30%"> <br />
</p><br><center>图1：RNN示意图</center></br>

<br>

近年来随着深度学习的发展，模型参数的数量飞速增长。为了训练这些参数，需要更大的数据集来避免过拟合。然而，对于大部分NLP任务来说，构建大规模的标注数据集非常困难（成本过高），特别是对于句法和语义相关的任务。相比之下，大规模的未标注语料库的构建则相对容易。为了利用这些数据，我们可以先从其中学习到一个好的表示，再将这些表示应用到其他任务中。最近的研究表明，基于大规模未标注语料库的预训练模型（Pretrained Models, PTM) 在NLP任务上取得了很好的表现。

近年来，大量的研究表明基于大型语料库的预训练模型（Pretrained Models, PTM）可以学习通用的语言表示，有利于下游NLP任务，同时能够避免从零开始训练模型。随着计算能力的发展，深度模型的出现（即 Transformer）和训练技巧的增强使得 PTM 不断发展，由浅变深。


<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/327f44ff3ed24493adca5ddc4dc24bf61eebe67c84a6492f872406f464fde91e" width="60%" height="50%"> <br />
</p><br><center>图2：预训练模型一览，图片来源于：https://github.com/thunlp/PLMpapers</center></br>
                                                                       
                                                                       
在此，使用ERNIE预训练模型完成序列标注任务。

## 三、数据处理

可见训练集由已标注数据和未标注数据组成


### 1.数据查看

In [ ]:
!head data/train.txt

In [ ]:
!awk '{print NR}' data/train.txt | tail -n1

### 2.数据格式调整
PaddleNLP格式为一条记录一条记录的样式，在此需要对原始数据进行格式调整。

In [ ]:
import os


def format_data(source_filename, target_filename):
    datalist = []
    with open(source_filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    words = ''
    labels = ''
    word=''
    label=''
    flag = 0
    cnt = 0
    for line in lines:
        if line == '\n':
            item = words + '\t' + labels + '\n'
            datalist.append(item)
            words = ''
            labels = ''
            flag = 0
            continue
        elif line == '  O\n':
            word = ' '
            label = 'O'
        elif '  I-'in line:
            word = ' '
            label = line.strip('\n').strip(' ')
        else:
            word, label = line.strip('\n').split(' ')

        # 使用\002是为了区分空格
        if flag == 1:
            words = words + '\002' + word
            labels = labels + '\002' + label
        else:
            words = words + word
            labels = labels + label
            flag = 1
    # 添加最后一行数据
    # item = words + '\t' + labels + '\n'
    # datalist.append(item)
    with open(target_filename, 'w', encoding='utf-8') as f:
        lines = f.writelines(datalist)
    print(f'{source_filename}文件格式转换完毕，保存为{target_filename}')

In [ ]:
format_data('data/train.txt', 'train.txt')

In [ ]:
!tail -n3 train.txt

In [ ]:
!awk '{print NR}' train.txt | tail -n1

### 3.数据集划分
按8:2进行分割，`-n`后面的数目具体由数据集的大小决定：

In [ ]:
#  训练集
!head -n65000 train.txt >train_converted.txt

In [ ]:
# 后 测试集
!tail -n15000 train.txt >val_converted.txt

### 4.PaddleNLP环境准备

In [ ]:
!pip install -U pip --user >log.log
!pip install --upgrade paddlenlp >log.log

In [ ]:
from functools import partial

import paddle
from paddlenlp.datasets import MapDataset
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.transformers import ErnieTokenizer, ErnieForTokenClassification
from paddlenlp.metrics import ChunkEvaluator
from utils import convert_example, evaluate, predict, load_dict

### 5.加载自定义数据集

In [ ]:
def load_dataset(datafiles):
    def read(data_path):
        with open(data_path, 'r', encoding='utf-8') as fp:
            for line in fp.readlines():
                words, labels = line.strip('\n').split('\t')
                words = words.split('\002')
                labels = labels.split('\002')
                yield words, labels

    if isinstance(datafiles, str):
        return MapDataset(list(read(datafiles)))
    elif isinstance(datafiles, list) or isinstance(datafiles, tuple):
        return [MapDataset(list(read(datafile))) for datafile in datafiles]   

In [ ]:
# 创建dataset
train_ds, dev_ds = load_dataset(datafiles=(
        './train_converted.txt', './val_converted.txt'))

In [ ]:
for i in range(5):
    print(train_ds[i])

### 6.label标签表构建
每条数据包含一句文本和这个文本中每个汉字以及数字对应的label标签

In [ ]:
# 生成标签
def gernate_dic(source_filename1,  target_filename):
    data_list=[]

    with open(source_filename1, 'r', encoding='utf-8') as f:
        lines=f.readlines()

    for line in lines:
        if line == '\n':
            continue
        elif line == '  O\n':
            dic = 'O'
        elif '  I-'in line:
            dic = line.strip('\n').strip(' ')
        else:
            dic = line.strip('\n').split(' ')[-1]
        if dic+'\n' not in data_list:
            data_list.append(dic+'\n')    

    with open(target_filename, 'w', encoding='utf-8') as f:
        lines=f.writelines(data_list)    

In [ ]:
# 生成dic
gernate_dic('data/train.txt', 'mytag.dic')

In [ ]:
# 查看生成的dic文件
!cat mytag.dic

### 7.数据处理
预训练模型ERNIE对中文数据的处理是以字为单位。PaddleNLP对于各种预训练模型已经内置了相应的tokenizer。指定想要使用的模型名字即可加载对应的tokenizer。

tokenizer作用为将原始输入文本转化成模型model可以接受的输入数据形式。

![https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_1.png](https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_1.png)

![https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_2.png](https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_2.png)

In [ ]:
label_vocab = load_dict('mytag.dic')
tokenizer = ErnieTokenizer.from_pretrained('ernie-1.0')

trans_func = partial(convert_example, tokenizer=tokenizer, label_vocab=label_vocab)

train_ds.map(trans_func)
dev_ds.map(trans_func)
print (train_ds[0])

**数据读入**

使用paddle.io.DataLoader接口多线程异步加载数据。

In [ ]:
ignore_label = -1
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack(),  # seq_len
    Pad(axis=0, pad_val=ignore_label)  # labels
): fn(samples)

train_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_size=256,
    return_list=True,
    collate_fn=batchify_fn)
dev_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_size=64,
    return_list=True,
    collate_fn=batchify_fn)

## 四、PaddleNLP一键加载预训练模型

### 1.加载预训练模型

快递单信息抽取本质是一个序列标注任务，PaddleNLP对于各种预训练模型已经内置了对于下游任务文本分类Fine-tune网络。以下教程以ERNIE为预训练模型完成序列标注任务。

`paddlenlp.transformers.ErnieForTokenClassification()`一行代码即可加载预训练模型ERNIE用于序列标注任务的fine-tune网络。其在ERNIE模型后拼接上一个全连接网络进行分类。

`paddlenlp.transformers.ErnieForTokenClassification.from_pretrained()`方法只需指定想要使用的模型名称和文本分类的类别数即可完成定义模型网络。

In [ ]:
# Define the model netword and its loss
model = ErnieForTokenClassification.from_pretrained("ernie-1.0", num_classes=len(label_vocab))

PaddleNLP不仅支持ERNIE预训练模型，还支持BERT、RoBERTa、Electra等预训练模型。
下表汇总了目前PaddleNLP支持的各类预训练模型。您可以使用PaddleNLP提供的模型，完成文本分类、序列标注、问答等任务。同时我们提供了众多预训练模型的参数权重供用户使用，其中包含了二十多种中文语言模型的预训练权重。中文的预训练模型有`bert-base-chinese, bert-wwm-chinese, bert-wwm-ext-chinese, ernie-1.0, ernie-tiny, gpt2-base-cn, roberta-wwm-ext, roberta-wwm-ext-large, rbt3, rbtl3, chinese-electra-base, chinese-electra-small, chinese-xlnet-base, chinese-xlnet-mid, chinese-xlnet-large, unified_transformer-12L-cn, unified_transformer-12L-cn-luge`等。

更多预训练模型参考：[PaddleNLP Transformer API](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/transformers.md)。

更多预训练模型fine-tune下游任务使用方法，请参考：[examples](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples)。

### 2.超参设置

In [ ]:
metric = ChunkEvaluator(label_list=label_vocab.keys(), suffix=True)
loss_fn = paddle.nn.loss.CrossEntropyLoss(ignore_index=ignore_label)
optimizer = paddle.optimizer.AdamW(learning_rate=1.28e-4, parameters=model.parameters())

## 五、模型训练与评估

### 1.训练模型

模型训练的过程通常有以下步骤：

1. 从dataloader中取出一个batch data
2. 将batch data喂给model，做前向计算
3. 将前向计算结果传给损失函数，计算loss。将前向计算结果传给评价方法，计算评价指标。
4. loss反向回传，更新梯度。重复以上步骤。

每训练一个epoch时，程序将会评估一次，评估当前模型训练的效果。

In [ ]:
step = 0
for epoch in range(10):
    for idx, (input_ids, token_type_ids, length, labels) in enumerate(train_loader):
        logits = model(input_ids, token_type_ids)
        loss = paddle.mean(loss_fn(logits, labels))
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        step += 1
        print("epoch:%d - step:%d - loss: %f" % (epoch, step, loss))
    evaluate(model, metric, dev_loader)

    paddle.save(model.state_dict(),
                './checkpoint/model_%d.pdparams' % step)

### 2.模型保存

In [ ]:
!mkdir ernie_result
model.save_pretrained('./ernie_result')
tokenizer.save_pretrained('./ernie_result')

## 六、预测

###  1.重启notebook，释放显存

重启notebook，释放显存，开始训练。

训练保存好的训练，即可用于预测。如以下示例代码自定义预测数据，调用`predict()`函数即可一键预测。

In [1]:
import numpy as np
import paddle
from paddle.io import DataLoader
import paddlenlp as ppnlp
from paddlenlp.datasets import load_dataset
from paddlenlp.data import Stack, Tuple, Pad, Dict
from paddlenlp.datasets import MapDataset
from paddlenlp.transformers import ErnieTokenizer, ErnieForTokenClassification
from paddlenlp.metrics import ChunkEvaluator
from utils import convert_example, evaluate, predict, load_dict
from functools import partial

### 2.导入预测数据

In [2]:
!head -n20 data/sample_per_line_preliminary_B.txt

### 3.定义test数据集

In [3]:
def load_dataset(datafiles):
    def read(data_path):
        with open(data_path, 'r', encoding='utf-8') as fp:
            # next(fp)  # 没有header，不用Skip header
            for line in fp.readlines():
                words = line.strip('\n')
                words=[ch for ch in words]
                labels=['O' for x in range(0,len(words))]

                yield words, labels
                # yield words

    if isinstance(datafiles, str):
        return MapDataset(list(read(datafiles)))
    elif isinstance(datafiles, list) or isinstance(datafiles, tuple):
        return [MapDataset(list(read(datafile))) for datafile in datafiles]      

In [4]:
test_ds = load_dataset(datafiles=('./data/sample_per_line_preliminary_B.txt'))

In [5]:
for i in range(20):
    print(test_ds[i])

### 4.加载最佳模型

In [6]:
label_vocab = load_dict('./mytag.dic')
tokenizer = ErnieTokenizer.from_pretrained('ernie-1.0')

trans_func = partial(convert_example, tokenizer=tokenizer, label_vocab=label_vocab)
test_ds.map(trans_func)
print (test_ds[0])

In [7]:
ignore_label = 1
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack(),  # seq_len
    Pad(axis=0, pad_val=ignore_label)  # labels
): fn(samples)

In [8]:
test_loader = paddle.io.DataLoader(
    dataset=test_ds,
    batch_size=30,
    return_list=True,
    collate_fn=batchify_fn)

In [9]:
def my_predict(model, data_loader, ds, label_vocab):
    pred_list = []
    len_list = []
    for input_ids, seg_ids, lens, labels in data_loader:
        logits = model(input_ids, seg_ids)
        # print(len(logits[0]))
        pred = paddle.argmax(logits, axis=-1)
        pred_list.append(pred.numpy())
        len_list.append(lens.numpy())
    preds ,tags= parse_decodes(ds, pred_list, len_list, label_vocab)
    return preds, tags

In [10]:
model = ErnieForTokenClassification.from_pretrained("ernie-1.0", num_classes=len(label_vocab))

In [11]:
# 选择目标checkpoint
model_dict = paddle.load('checkpoint/model_2540.pdparams')
model.set_dict(model_dict)

### 5.预测并保存

In [12]:
from utils import *
preds, tags = my_predict(model, test_loader, test_ds, label_vocab)

In [ ]:
print(tags[:100])

In [13]:
file_path = "ernie_results.txt"
with open(file_path, "w", encoding="utf8") as fout:
    fout.write("\n".join(preds))
# Print some examples
print(
    "The results have been saved in the file: %s, some examples are shown below: "
    % file_path)

In [14]:
!head ernie_results.txt

In [ ]:
# 调整数据格式
wp = open('data/sample_per_line_preliminary_B.txt', 'r', encoding='utf-8')
lp = open('./ernie_results.txt', 'r', encoding='utf-8')
with open('result.txt', 'w', encoding='utf-8') as f:
    word_line = wp.readlines()
    label_line = lp.readlines()
    print('word_line:{},label_line {}'.format(len(word_line), len(label_line)))
    for i in range(len(word_line)):
        labels = label_line[i].strip('\n').strip(' ').split(' ')
        words = word_line[i].strip('\n')
        print('i:{} labels:{},words {}'.format(i, len(labels), len(words)))
        for j in range(len(labels)):
            f.write(words[j] + ' ' + labels[j] + '\n')
        f.write('\n')
wp.close()
lp.close()